## Utilidades previas

In [17]:
#Se importan las librerías necesarias
import numpy as np
from hidden_markov import hmm
import pandas as pd
import os
import matplotlib.pyplot as plt

In [18]:
#Se carga el archivo que contiene los datos
dataFolder = './data/'
dataFile = 'prots-L30.txt'
dataFilepath = os.path.join(dataFolder,dataFile)
df = pd.read_csv(dataFilepath)
df.head()

,pdb_id,seq,sst3,len
0,1AI0,FVNQHLCGSHLVEALYLVCGERGFFYTPKT,CCCHHHHHHHHHHHHHHHHHHHCEEECCCC,30
1,1APH,FVNQHLCGSHLVEALYLVCGERGFFYTPKA,CCCCCCCCHHHHHHHHHHHHHHCEEECCCC,30
2,1B4G,MISSVCVSSYRGRKSGNKPPSKTCLKEEMA,CCCCCECCCCCCCCCCCCCCCCCCCCCCCC,30
3,1B9E,FVNQHLCGEHLVEALYLVCGERGFFYTPKT,CCCCCCCHHHHHHHHHHHHHHHCEEECCCC,30
4,1BH4,CGESCVWIPCISAALGCSCKNKVCYRNGIP,CCCCCCCCCCCHHHHCCCCCCCCCCCCCCC,30


In [19]:
#Se definen los 20 aminoácidos
alfabeto=[ele for ele in "RKDEQSCHNTWYMAILFVPG"]

### Vector inicial de probabilidades

In [20]:
numSecuencias = len(df['seq'])
lenAlfab = len(alfabeto)
#frecAbsoluPos1 = [0 for i in range(lenAlfab)]
Pini = np.zeros(lenAlfab).astype(float)

for nseq in range(numSecuencias):
    secuenciaTabla = df.loc[nseq,'seq']
    primerAminoacido = secuenciaTabla[0]
    posicionAlfabeto = alfabeto.index(primerAminoacido)
    Pini[posicionAlfabeto] += 1

frecuenciasAbsolutasIniciales = dict(zip(alfabeto,Pini))
frecuenciasAbsolutasIniciales

{'R': 2.0,
 'K': 10.0,
 'D': 6.0,
 'E': 8.0,
 'Q': 3.0,
 'S': 8.0,
 'C': 9.0,
 'H': 3.0,
 'N': 3.0,
 'T': 3.0,
 'W': 0.0,
 'Y': 3.0,
 'M': 15.0,
 'A': 21.0,
 'I': 1.0,
 'L': 2.0,
 'F': 39.0,
 'V': 9.0,
 'P': 5.0,
 'G': 24.0}

In [21]:
#Vector inicial de probabilidades

print('Suma de Pini: {}\nNúmero de secuencias: {}'.format(sum(Pini),numSecuencias))
sum(Pini),numSecuencias
# Frecuencias Relativas
sumFrecuenciasAbsolutas = sum(Pini)
Pini = Pini/sumFrecuenciasAbsolutas
print('\n')
print(Pini,Pini.sum())
frecuenciasRelativasIniciales = dict(zip(alfabeto,Pini))
print('\n')
print(frecuenciasRelativasIniciales)

Suma de Pini: 174.0
Número de secuencias: 174


[0.01149425 0.05747126 0.03448276 0.04597701 0.01724138 0.04597701
 0.05172414 0.01724138 0.01724138 0.01724138 0.         0.01724138
 0.0862069  0.12068966 0.00574713 0.01149425 0.22413793 0.05172414
 0.02873563 0.13793103] 1.0


{'R': 0.011494252873563218, 'K': 0.05747126436781609, 'D': 0.034482758620689655, 'E': 0.04597701149425287, 'Q': 0.017241379310344827, 'S': 0.04597701149425287, 'C': 0.05172413793103448, 'H': 0.017241379310344827, 'N': 0.017241379310344827, 'T': 0.017241379310344827, 'W': 0.0, 'Y': 0.017241379310344827, 'M': 0.08620689655172414, 'A': 0.1206896551724138, 'I': 0.005747126436781609, 'L': 0.011494252873563218, 'F': 0.22413793103448276, 'V': 0.05172413793103448, 'P': 0.028735632183908046, 'G': 0.13793103448275862}


### Matriz de transiciones

In [22]:
#Matriz de transiciones con valores absolutos

columns = alfabeto
rows = alfabeto
MT = pd.DataFrame(0.0,columns=columns,index=rows) # inicializamos como float
for nseq in range(df.shape[0]):
    secuencia = df.loc[nseq,'seq']
    for naa in range(len(secuencia)-1): # recorremos hasta el penultimo
        pos1 = secuencia[naa]
        pos2 = secuencia[naa+1]
        MT.loc[pos1,pos2]+=1
MT

,R,K,D,E,Q,S,C,H,N,T,W,Y,M,A,I,L,F,V,P,G
R,32.0,14.0,6.0,10.0,9.0,6.0,12.0,5.0,7.0,9.0,3.0,17.0,2.0,18.0,24.0,29.0,5.0,4.0,15.0,52.0
K,17.0,27.0,7.0,21.0,10.0,20.0,17.0,10.0,12.0,49.0,3.0,6.0,1.0,18.0,9.0,13.0,2.0,11.0,23.0,12.0
D,4.0,9.0,15.0,6.0,4.0,16.0,12.0,4.0,5.0,5.0,2.0,5.0,1.0,7.0,7.0,17.0,3.0,9.0,12.0,13.0
E,71.0,29.0,16.0,16.0,5.0,11.0,7.0,3.0,8.0,7.0,2.0,8.0,1.0,52.0,14.0,15.0,4.0,9.0,4.0,13.0
Q,9.0,11.0,4.0,8.0,6.0,3.0,12.0,35.0,4.0,4.0,1.0,12.0,0.0,12.0,10.0,8.0,1.0,3.0,3.0,18.0
S,10.0,27.0,20.0,10.0,9.0,21.0,35.0,34.0,11.0,8.0,7.0,12.0,3.0,12.0,8.0,17.0,5.0,9.0,12.0,25.0
C,26.0,24.0,14.0,10.0,12.0,33.0,36.0,0.0,5.0,24.0,1.0,25.0,4.0,24.0,43.0,8.0,2.0,19.0,16.0,89.0
H,1.0,4.0,1.0,3.0,1.0,12.0,7.0,6.0,1.0,2.0,0.0,2.0,4.0,5.0,7.0,68.0,1.0,5.0,0.0,2.0
N,15.0,5.0,4.0,4.0,43.0,4.0,6.0,0.0,6.0,8.0,2.0,2.0,0.0,6.0,7.0,18.0,2.0,4.0,5.0,10.0
T,8.0,30.0,6.0,9.0,1.0,13.0,34.0,5.0,4.0,8.0,2.0,9.0,6.0,14.0,7.0,11.0,8.0,13.0,32.0,9.0


In [23]:
#Matriz de transiciones con valores normalizados

for row in MT.index:
    fila = MT.loc[row,:]
    filaSum = fila.sum()
    MT.loc[row,:] = MT.loc[row,:]/filaSum
MT

,R,K,D,E,Q,S,C,H,N,T,W,Y,M,A,I,L,F,V,P,G
R,0.114695,0.050179,0.021505,0.035842,0.032258,0.021505,0.043011,0.017921,0.025090,0.032258,0.010753,0.060932,0.007168,0.064516,0.086022,0.103943,0.017921,0.014337,0.053763,0.186380
K,0.059028,0.093750,0.024306,0.072917,0.034722,0.069444,0.059028,0.034722,0.041667,0.170139,0.010417,0.020833,0.003472,0.062500,0.031250,0.045139,0.006944,0.038194,0.079861,0.041667
D,0.025641,0.057692,0.096154,0.038462,0.025641,0.102564,0.076923,0.025641,0.032051,0.032051,0.012821,0.032051,0.006410,0.044872,0.044872,0.108974,0.019231,0.057692,0.076923,0.083333
E,0.240678,0.098305,0.054237,0.054237,0.016949,0.037288,0.023729,0.010169,0.027119,0.023729,0.006780,0.027119,0.003390,0.176271,0.047458,0.050847,0.013559,0.030508,0.013559,0.044068
Q,0.054878,0.067073,0.024390,0.048780,0.036585,0.018293,0.073171,0.213415,0.024390,0.024390,0.006098,0.073171,0.000000,0.073171,0.060976,0.048780,0.006098,0.018293,0.018293,0.109756
S,0.033898,0.091525,0.067797,0.033898,0.030508,0.071186,0.118644,0.115254,0.037288,0.027119,0.023729,0.040678,0.010169,0.040678,0.027119,0.057627,0.016949,0.030508,0.040678,0.084746
C,0.062651,0.057831,0.033735,0.024096,0.028916,0.079518,0.086747,0.000000,0.012048,0.057831,0.002410,0.060241,0.009639,0.057831,0.103614,0.019277,0.004819,0.045783,0.038554,0.214458
H,0.007576,0.030303,0.007576,0.022727,0.007576,0.090909,0.053030,0.045455,0.007576,0.015152,0.000000,0.015152,0.030303,0.037879,0.053030,0.515152,0.007576,0.037879,0.000000,0.015152
N,0.099338,0.033113,0.026490,0.026490,0.284768,0.026490,0.039735,0.000000,0.039735,0.052980,0.013245,0.013245,0.000000,0.039735,0.046358,0.119205,0.013245,0.026490,0.033113,0.066225
T,0.034934,0.131004,0.026201,0.039301,0.004367,0.056769,0.148472,0.021834,0.017467,0.034934,0.008734,0.039301,0.026201,0.061135,0.030568,0.048035,0.034934,0.056769,0.139738,0.039301


## Secuencia más probable en una cadena de Markov

In [24]:
#Se definen los parámetros del modelo
states=tuple(alfabeto)
obs=tuple("S") #solo se usa una observación para reducir la cantidad de cálculos
start_prob=np.asmatrix(Pini)
trans_prob=np.asmatrix(MT)
#La matriz de emisiones tiene dimension 20x1
em_prob= np.asmatrix(np.ones((20))).T  #la matriz de emisiones es una matriz con todos elementos 1, para evitar que se alteren las probabilidades a la hora de buscar la secuencia más probable

In [25]:
#Se define el modelo de Markov Oculto
mod_HMM = hmm(states=states,
             observations=obs,
             start_prob=start_prob,
             trans_prob=trans_prob,
             em_prob=em_prob)

In [26]:
#Se utiliza una cadena de observaciones de longitud 20 para poder aplicar el algoritmo de Viterbi
observ="SSSSSSSSSSSSSSSSSSSS"

In [30]:
#Se busca la secuencia más probable de aminoácidos
secmasprobable=''.join(mod_HMM.viterbi(observ))
secmasprobable

20

### Verosimilitud de la secuencia

In [28]:
def myloglik(vecProbInicial, matrizTransicion, alfa, secuencia):
    
    loglik = 0.0
    
    primeraLetra = secuencia[0]
    primeraPosicionEnAlfabeto = alfa.index(primeraLetra)
    
    loglik += np.log(vecProbInicial[primeraPosicionEnAlfabeto])
    
    for i in range(len(secuencia)-1):
        
        letraEntrada = secuencia[i]
        letraSalida = secuencia[i+1]
        
        posicionLetraEntradaAlfabeto = alfa.index(letraEntrada)
        posicionLetraSalidaAlfabeto = alfa.index(letraSalida)
        probabilidadTransicion = matrizTransicion.iloc[posicionLetraEntradaAlfabeto,
                                                 posicionLetraSalidaAlfabeto]
        loglik += np.log(probabilidadTransicion)
    return loglik

In [29]:
loglik = myloglik(Pini, MT, alfabeto, secmasprobable)
lik=np.exp(loglik)
print(secmasprobable,'\nLoglikelihood: {}\nLikelihood:: {}\n'.format(loglik,lik))

FVNQHLVNQHLVNQHLVNQH 
Loglikelihood: -28.906825684768148
Likelihood:: 2.7920623718947307e-13



###### Verosimilitudes de las cadenas obtenidas en el ejercicio 6 de la primera parte

In [16]:
secuencia1 = "SEQWENCIAVERQSMILYES"
loglik1 = myloglik(Pini, MT, alfabeto, secuencia1)
lik1=np.exp(loglik1)
print(secuencia1,'\nLoglikelihood: {}\nLikelihood:: {}\n'.format(loglik1,lik1))

secuencia2 = "FVCQHLVCQHLVCQHLVCQQ"
loglik2 = myloglik(Pini, MT, alfabeto, secuencia2)
lik2=np.exp(loglik2)
print(secuencia2,'\nLoglikelihood: {}\nLikelihood:: {}\n'.format(loglik2,lik2))

SEQWENCIAVERQSMILYES 
Loglikelihood: -64.0330953932442
Likelihood:: 1.5516008595950296e-28

FVCQHLVCQHLVCQHLVCQQ 
Loglikelihood: -38.54576794984071
Likelihood:: 1.8188030702995e-17



Al comparar la verosimilitud de la cadena obtenida con las verosimilitudes obtenidas en el ejercicio 6 de la primera parte, se observa que la cadena obtenida tiene una verosimilitud mucho más alta del orden de $10^{-13}$, mientras que las dos cadenas dadas tienen verosimilitudes del orden de $10^{-28}$ y $10^{-17}$ respectivamente.

La secuencia obtenida con el modelo de Markov sin variables ocultas da información sobre la probabilidad que tiene cada aminoácido de aparecer en una posición específica en una proteína, dado el conjunto inicial de datos. También podría aportar información sobre la evolución de la secuencia de proteínas en el set de datos. 